In [0]:
import numpy as np
import pandas as pd

from google.colab import files
uploaded = files.upload()

Saving data17smaller_2.csv to data17smaller_2.csv


In [0]:
data18smaller = pd.read_csv("data17smaller_2.csv", low_memory = False, dtype = {'subsidies_flag':"int64"})
#data18smaller = data18smaller.iloc[:,0:8]
data18smaller.head()

,violations,landuse,bldgclass,residfar,yearbuilt,ownertype,subsidies_flag,unit_ratio
0,0,2,3,3,0,0,0,4
1,0,3,2,3,0,0,0,3
2,0,3,5,0,0,0,0,2
3,0,3,3,3,2,0,0,1
4,0,3,3,3,0,0,0,1


In [0]:
!pip install pgmpy==0.1.7
from pgmpy.estimators import HillClimbSearch, BicScore, BayesianEstimator
from pgmpy.models import BayesianModel
from sklearn.model_selection import train_test_split
from pgmpy.inference import VariableElimination

In [0]:
data18train, data18test = train_test_split(data18smaller,random_state=9,test_size=0.2)
hc = HillClimbSearch(data18train, scoring_method=BicScore(data18train))
best_model = hc.estimate()
print(best_model.edges())

[('bldgclass', 'residfar'), ('bldgclass', 'yearbuilt'), ('bldgclass', 'violations'), ('bldgclass', 'ownertype'), ('bldgclass', 'subsidies_flag'), ('subsidies_flag', 'affordability_flag'), ('residfar', 'violations'), ('residfar', 'subsidies_flag'), ('violations', 'yearbuilt'), ('violations', 'subsidies_flag'), ('violations', 'ownertype'), ('violations', 'unitsres'), ('violations', 'affordability_flag'), ('landuse', 'residfar'), ('landuse', 'violations'), ('landuse', 'unitsres'), ('landuse', 'bldgclass')]


In [0]:
best_model.fit(data18train, estimator=BayesianEstimator, prior_type='K2')

for cpd in best_model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)
    
print best_model.local_independencies('violations')

for thenode in best_model.nodes():
    print best_model.active_trail_nodes(thenode)

CPD of unitsres:
+-------------+------------------------+----------------------+------------------------+-----------------------+------------------------+----------------------+------------------------+-----------------------+---------------+---------------+
| landuse     | landuse(0)             | landuse(0)           | landuse(1)             | landuse(1)            | landuse(2)             | landuse(2)           | landuse(3)             | landuse(3)            | landuse(4)    | landuse(4)    |
+-------------+------------------------+----------------------+------------------------+-----------------------+------------------------+----------------------+------------------------+-----------------------+---------------+---------------+
| violations  | violations(0)          | violations(1)        | violations(0)          | violations(1)         | violations(0)          | violations(1)        | violations(0)          | violations(1)         | violations(0) | violations(1) |
+-------------+

In [0]:
from collections import defaultdict
from pgmpy.inference import VariableElimination
def temp_predict(model,data):
    missing_variables = list(set(model.nodes()) - set(data.columns))
    pred_values = defaultdict(list)
    model_inference = VariableElimination(model)
    count = 0
    for index, data_point in data.iterrows():
        count = count+1
        if count %1000 == 1:
          print count
        states_dict = model_inference.map_query(variables=missing_variables, evidence=data_point.astype(int).to_dict())
        for k, v in states_dict.items():
            pred_values[k].append(v)
    return pd.DataFrame(pred_values, index=data.index)

In [0]:
best_model.fit(data18train, estimator=BayesianEstimator, prior_type='K2')
test_x = data18test.drop(columns=['violations'])
predicted_test = temp_predict(best_model, test_x)
print "Out of sample:",(data18test.loc[:,'violations']==predicted_test['violations']).mean()

Out of sample: 0.8921727776405406


In [0]:

train_x = data18train.drop(columns=['violations'])
predicted_train = temp_predict(best_model, train_x)
print "In of sample:",(data18train.loc[:,'violations']==predicted_train['violations']).mean()

In [0]:
data18train, data18test = train_test_split(data18smaller,random_state=9,test_size=0.2)
hc = HillClimbSearch(data18train, scoring_method=BicScore(data18train))
best_model = hc.estimate()
print(best_model.edges())

[('bldgclass', 'subsidies_flag'), ('bldgclass', 'ownertype'), ('bldgclass', 'residfar'), ('bldgclass', 'yearbuilt'), ('bldgclass', 'unit_ratio'), ('bldgclass', 'violations'), ('residfar', 'unit_ratio'), ('residfar', 'violations'), ('residfar', 'yearbuilt'), ('yearbuilt', 'unit_ratio'), ('yearbuilt', 'violations'), ('yearbuilt', 'ownertype'), ('yearbuilt', 'subsidies_flag'), ('violations', 'subsidies_flag'), ('landuse', 'residfar'), ('landuse', 'bldgclass')]


In [0]:
best_model.fit(data18train, estimator=BayesianEstimator, prior_type='K2')

for cpd in best_model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)
    
print best_model.local_independencies('violations')

for thenode in best_model.nodes():
    print best_model.active_trail_nodes(thenode)

CPD of bldgclass:
+--------------+------------------------+-----------------------+-----------------------+------------------------+
| landuse      | landuse(0)             | landuse(1)            | landuse(2)            | landuse(3)             |
+--------------+------------------------+-----------------------+-----------------------+------------------------+
| bldgclass(0) | 0.2704541235818875     | 1.059411814560556e-05 | 8.397715821296607e-05 | 3.469571854833114e-05  |
+--------------+------------------------+-----------------------+-----------------------+------------------------+
| bldgclass(1) | 0.7295022001899916     | 1.059411814560556e-05 | 8.397715821296607e-05 | 3.469571854833114e-05  |
+--------------+------------------------+-----------------------+-----------------------+------------------------+
| bldgclass(2) | 1.0919057030234869e-05 | 0.9558013390965336    | 8.397715821296607e-05 | 0.23194087849559364    |
+--------------+------------------------+---------------------

In [0]:
best_model.fit(data18train, estimator=BayesianEstimator, prior_type='K2')
test_x = data18test.drop(columns=['violations'])
predicted_test = temp_predict(best_model, test_x)
print "Out of sample:",(data18test.loc[:,'violations']==predicted_test['violations']).mean()

1
1001
2001
3001
4001
5001
6001
7001
8001
9001
10001
11001
12001
13001
14001
15001
16001
17001
18001
19001
20001
21001
22001
23001
24001
25001
26001
27001
28001
29001
30001
31001
32001
33001
34001
35001
36001
37001
38001
39001
40001
41001
42001
43001
44001
45001
46001
47001
48001
49001
50001
51001
52001
53001
54001
55001
56001
Out of sample: 0.8934199149476805
